In [ ]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

In [ ]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

In [ ]:
# !pip install datasets
import datasets

In [ ]:
from datasets import Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

# Response Section
a.  What are the cosine similarity scores for the following pairs: 
-  plane / car 
-  planet / sun 
-  cup / article 
-  sugar / approach

b.  What is the value of the Spearman correlation coefficients computed in Step 4?

c.  How do you interpret each coefficient value with respect to the word similarity task? 
Are the coefficient values for the two vector space models you created different, and 
if so, why?

d.  What is the value of the Spearman correlation coefficient and how do you interpret it?

e.  Create a table of results that summarises your experiments. Tips: In case you have 
run several experiments with different hyperparameters choose to present the most 
significant. It might be worth presenting more than one experiment with only a single 
hyperparameter change if you want to emphasise a striking difference worth 
discussing. Finally, apart from the Spearman correlation coefficients, make sure you 
also include the most significant hyperparameters as separate columns (for example, 
see Table 2 from Merity et al., 2016).

f.  Using the table of results from your answer to question g., write a short discussion 
section that answers the following questions:  
i.  Does a bigger corpus yield better representations?   
ii.  Does a bigger vocabulary yield better representations?  
iii.  Do bigger word vectors yield better representations?  
iv.  Does a bigger context window yield better representations? 
v.  Step 6 requires you to look for the best combination of hyperparameters using 
the same two datasets for evaluation. Is this a good practice?

g.  What are the top-5 analogies for the following configurations: 
-  man is to woman as king is to ___? 
-  Athens is to Greece as Rome is to ___? 
-  reading is to read as playing is to ___? 
-  Greece is to souvlaki as Italy is to ___? 
-  airplane is to propeller as car is to ___? 
Is the top-1 answer always the “correct”? What about the rest of the results?

h.  What are the top-5 analogies for the following configurations? Can you identify any 
gender-based stereotypes? Briefly discuss your findings: 
-  man is to woman as computer programmer is to ___? 
-  man is to woman as superstar is to ___? 
-  man is to woman as guitarist is to ___? 
-  man is to woman as boss is to ___?